In [4]:
import numpy as np
import networkx as nx
from tqdm import tqdm
from utils.deepwalk import deepwalk
path = './data/'

# Node Embedding using Deep Walk

In [ ]:
# load the authors papers dictionary
author_paper = np.load(path+'author_paper.npy',allow_pickle='TRUE').item()
# load the graph    
G = nx.read_edgelist(path+'collaboration_network.edgelist', delimiter=' ', nodetype=int)
n_nodes = G.number_of_nodes()
n_edges = G.number_of_edges()

In [ ]:
# Generate the weights of the graph:  each weight defines the number of papers in which 2 authors co-authored.
G_weigthed = G.copy()
for i,edge in enumerate(tqdm(list(G.edges))):
    source,target = edge
    paper_author_1 = author_paper[str(int(source))]
    paper_author_2 = author_paper[str(int(target))]
    count = len(list(set(paper_author_1).intersection(set(paper_author_2))))
    if count>0:
        G_weigthed[source][target]['weight'] = count
    else: 
        # We note that there are some edges between authors without having any common papers Id in the author_papers.txt.
        G_weigthed[source][target]['weight'] = 1 

100%|██████████| 1777338/1777338 [00:09<00:00, 178748.13it/s]


In [ ]:
n_dim = 256
n_walks = 10
walk_length = 20
weighted  = False #if True, our deep walk algorithm considers a weighted graph 

if weighted == True:
    model = deepwalk(G_weigthed, n_walks, walk_length, n_dim,weighted=True)
else:
    model = deepwalk(G, n_walks, walk_length, n_dim,weighted=False)
#Compute the node embedding:
embeddings = np.zeros((n_nodes, n_dim +1))
for i, node in enumerate(G.nodes()):
    embeddings[i,0] = str(node)
    embeddings[i,1:] = model.wv[str(node)]
#Save the node embedding:
pd.DataFrame(embeddings).to_csv(path+'node_embd_DW_Weighted_256.csv',index=False)